In [1]:
import numpy as np

from onix.units import ureg
from onix.experiments.exp_executor import ExpSequence

In [37]:
# minics what happens in the experiment definition creator
# scans lf detuning from -10 kHz to 10 kHz.

parameters = {
    "hyperfine": {
        "7F0": {
            "a": 0 * ureg.MHz,
            "b": 119.2 * ureg.MHz,
            "c": 209.25 * ureg.MHz,
        },
        "5D0": {
            "a": 450.8 * ureg.MHz,
            "b": 191.3 * ureg.MHz,
            "c": 0 * ureg.MHz,
        }
    },
    "ao": {
        "channel_name": "ao_dp",  # links to the actual hardware channel.
        "order": 2,  # double-pass, upshifting.
        "frequency_ac": 80 * ureg.MHz,  # ao frequency of the a -> c' transition.
    },
    "shutter": {
        "channel_name": "shutter",
    },
    "field_plate": {
        "channel_name": "field_plate",
        "low_voltage": 1.25 * ureg.V,  # AWG output before the HV amplifier
        "high_voltage": -0.002 * ureg.V,  # compensates the dc offset.
        "field_to_voltage": 1 / (0.8 * ureg.cm),
        "dipole_moment": 27.7 * ureg.kHz / (ureg.V / ureg.cm),
        "ramp_up_time": 3 * ureg.ms,  # ramp time from low to high.
        "high_wait_time": 2 * ureg.ms,  # wait time in the high state before voltage is stabilized.
        "ramp_down_time": 0 * ureg.ms,  # ramp time from high to low.
        "low_wait_time": 2 * ureg.ms,  # wait time in the low state before voltage is stabilized.
        "use": True,  # uses the field plate in the experiment.
        "during_detect": True,  # if True, the field plate is on during chasm and detect. Otherwise, it is on during antihole.
    },
    "chasm": {
        "transitions": "ac",  # only burns a chasm on a -> c'
        "detunings": 0 * ureg.MHz,
        "scans": 3 * ureg.MHz,  # scans +/-3 MHz in the optical frequency.
        "amplitudes": 2000,
        "durations": 1 * ureg.ms,
        "repeats": 25,
    },
    "lf": {
        "channel_name": "rf_coil",
        "rabi": {
            "center_frequency": 140 * ureg.kHz,
            "Sigma": 0,  # 0, +1, -1.
            "Zeeman_shift_along_b": 5 * ureg.kHz,
            "detuning": 0 * ureg.kHz,
            "amplitude": 1000,
            "duration": 1 * ureg.ms,
        },
    },
}

detuning_scan = np.linspace(-10, 10, 21) * ureg.kHz

additional_sequence_steps = {}
for kk in range(len(detuning_scan)):
    if kk > 0:
        override_parameters = {"lf": {"rabi": {"detuning": detuning_scan[kk]}}}
        additional_sequence_steps[f"lf_rabi_{kk}"] = override_parameters

In [40]:
additional_sequence_steps

{'lf_rabi_1': {'lf': {'rabi': {'detuning': -9.0 <Unit('kilohertz')>}}},
 'lf_rabi_2': {'lf': {'rabi': {'detuning': -8.0 <Unit('kilohertz')>}}},
 'lf_rabi_3': {'lf': {'rabi': {'detuning': -7.0 <Unit('kilohertz')>}}},
 'lf_rabi_4': {'lf': {'rabi': {'detuning': -6.0 <Unit('kilohertz')>}}},
 'lf_rabi_5': {'lf': {'rabi': {'detuning': -5.0 <Unit('kilohertz')>}}},
 'lf_rabi_6': {'lf': {'rabi': {'detuning': -4.0 <Unit('kilohertz')>}}},
 'lf_rabi_7': {'lf': {'rabi': {'detuning': -3.0 <Unit('kilohertz')>}}},
 'lf_rabi_8': {'lf': {'rabi': {'detuning': -2.0 <Unit('kilohertz')>}}},
 'lf_rabi_9': {'lf': {'rabi': {'detuning': -1.0 <Unit('kilohertz')>}}},
 'lf_rabi_10': {'lf': {'rabi': {'detuning': 0.0 <Unit('kilohertz')>}}},
 'lf_rabi_11': {'lf': {'rabi': {'detuning': 1.0 <Unit('kilohertz')>}}},
 'lf_rabi_12': {'lf': {'rabi': {'detuning': 2.0 <Unit('kilohertz')>}}},
 'lf_rabi_13': {'lf': {'rabi': {'detuning': 3.0 <Unit('kilohertz')>}}},
 'lf_rabi_14': {'lf': {'rabi': {'detuning': 4.0 <Unit('kilohertz

In [38]:
# The ExpSequence object should be built by the ExpExecutor object automatically when a new experiment definition file is found.
# Here we manually build the ExpSequence objects for the example.
exp_seqs = []
for kk in range(len(detuning_scan)):
    parameters["lf"]["rabi"]["detuning"] = detuning_scan[kk]  # this step does not change the sequence being run. It ensures that the correct parameters after overriden is saved.
    if kk == 0:
        exp_seq = ExpSequence(
            exp_sequence = ["chasm", f"lf_rabi_{kk}"],
            parameters = parameters,
            additional_sequence_steps = additional_sequence_steps,
        )
        # the segments should be programmed on the AWG
        exp_seqs.append(exp_seq)
    else:
        exp_seq = ExpSequence(
            exp_sequence = ["chasm", f"lf_rabi_{kk}"],
            parameters = parameters,
        )
        # the segments should NOT be programmed on the AWG, since it can reuse the programmed segments from the first run.
        exp_seqs.append(exp_seq)

In [39]:
# confirming that the frequencies are correct

for kk in range(len(detuning_scan)):
    for s in exp_seqs[0].segments:
        if s.name == f"lf_rabi_{kk}":
            print(kk, s._awg_pulses[6]._frequency)
            break

0 130.0 kilohertz
1 131.0 kilohertz
2 132.0 kilohertz
3 133.0 kilohertz
4 134.0 kilohertz
5 135.0 kilohertz
6 136.0 kilohertz
7 137.0 kilohertz
8 138.0 kilohertz
9 139.0 kilohertz
10 140.0 kilohertz
11 141.0 kilohertz
12 142.0 kilohertz
13 143.0 kilohertz
14 144.0 kilohertz
15 145.0 kilohertz
16 146.0 kilohertz
17 147.0 kilohertz
18 148.0 kilohertz
19 149.0 kilohertz
20 150.0 kilohertz
